First we need to import used libraries:

In [49]:
import gym, itertools, numpy as np, pandas as pd, random

After that we can divide observation space values into certain ranges (cart position and pole angle are fixed, cart velocity and pole angular velocity need to be calculated by GA):

In [50]:
def mapping_observation_to_1D(observation, bits):
    rule = []
    lst_observations =[]
    #var_for_velocity is fixed value for now
    var_for_velocity = 5

    #given range (-2.4, 2.4)
    cart_position_range = np.linspace(-2.4, 2.4, num=bits+1)
    lst_observations.append(cart_position_range)
    #TODO
    cart_velocity_range = np.linspace(-var_for_velocity, var_for_velocity, num=bits+1)
    lst_observations.append(cart_velocity_range)
    #given range (-0.2095 rad, 0.2095 rad)
    pole_angle_range = np.linspace(-0.2095, 0.2095, num=bits+1)
    lst_observations.append(pole_angle_range)
    #TODO
    pole_angular_velocity = np.linspace(-var_for_velocity, var_for_velocity, num=bits+1)
    lst_observations.append(pole_angular_velocity)

    for observation_index in range(len(lst_observations)):
        for index in range(1, len(lst_observations[observation_index])):
            #lower bound value <= observed value <= upper bound value
            if lst_observations[observation_index][index-1] <= observation[observation_index] <= lst_observations[observation_index][index]:
                rule.append(1)
                #if we find it then fill the rest with 0s and break loop
                for i in range(len(lst_observations[observation_index])-index-1):
                    rule.append(0)
                break
            else:
                rule.append(0)
                
        #5 break cells between observation values
        for i in range(0,5):
            rule.append(0)
            
    return rule


Create list containing rulesets which map every combinations of 5 neighbours bits to randomly chosen 0 or 1:

In [51]:
def generate_ruleset(size):
    list_dict_rules = []
    combinations_5_neighbours = list(itertools.product([0, 1], repeat=5))
    combinations_5_neighbours = ["".join(str(seq)).replace(',','').replace(' ','').replace('(','').replace(')','') for seq in combinations_5_neighbours]

    for iteration in range(0,size):
        dict_rules = {}
        for combination in combinations_5_neighbours:
            dict_rules[combination] = random.choice([0,1])
        list_dict_rules.append(dict_rules)

    return list_dict_rules

We define function action to behave accordingly to previously generated ruleset:

In [52]:
def action(mapped_observation, ruleset):
    v = 0
    oldline = mapped_observation
    newline = [0] * len(mapped_observation)
    for x in range(0,len(mapped_observation)):
        for bit in range(len(oldline)):
            combination = str(oldline[bit % 60]) + str(oldline[(bit+1) % 60]) + str(oldline[(bit+2) % 60]) + str(oldline[(bit+3) % 60]) + str(oldline[(bit+4) % 60])
            newline[(bit+2) % 60] = ruleset[combination]
        oldline = newline

    v = 1 if newline.count(1) > (len(mapped_observation)/2) else 0
    return v

Prepare the environment and generate a given number of rulesets:

In [53]:
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset()
list_to_return = []

rulesets = generate_ruleset(100)

Run the simulation but remember to change a total number of episodes accrodingly:

In [54]:
for ruleset in rulesets:
    score = 0
    maxscore = 0
    dict_to_return = {}
    dict_to_return["Ruleset"] = ruleset.values()
    
    for episode in range(100):
        mapped_observation = mapping_observation_to_1D(observation, 10)
        observation, reward, terminated, truncated, info = env.step(action(mapped_observation, ruleset))
        score += reward
        
        if terminated or truncated:
            observation, info = env.reset()
            if score > maxscore:
                maxscore = score 
            score = 0

    dict_to_return["Maxscore"] = int(maxscore)
    list_to_return.append(dict_to_return)

env.close()


Results are saved into .csv file containing applied ruleset in the 1st column and maximum score for this ruleset in the 2nd column:

In [ ]:

df = pd.DataFrame(list_to_return)
# df = df.astype(str)
df.to_csv('results.csv', index=False, header=False, sep=";")